In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
data_file = open('data/CSE151B_groupchat_3mo.json')
json_data = json.load(data_file)
json_data['messages'][1]['content'].encode('latin1').decode('utf-8')

"Can one of you guys download your facebook messenger data and upload the json file for this groupchat to the github repo? I can't download the 3 month one for some reason"

In [3]:
one_minute = 60000

In [4]:
len(json_data['messages'])

1664

In [30]:
prompt_response = []
for i in range(len(json_data['messages'])-1):
    this_dict = {}
    time_difference = json_data['messages'][i+1]['timestamp_ms'] - json_data['messages'][i]['timestamp_ms']
    
    #Keeps responses within 2minutes of previous message, removes photos
    if json_data['messages'][i]['sender_name'] != json_data['messages'][i+1]['sender_name'] and \
        time_difference <= one_minute*2 and 'content' in json_data['messages'][i] and \
        'content' in json_data['messages'][i+1]:
        
        this_dict['prompt'] = json_data['messages'][i]['content'].encode('latin1').decode('utf-8').replace('\n', ' ')
        this_dict['response'] = json_data['messages'][i+1]['content'].encode('latin1').decode('utf-8').replace('\n', ' ')
        this_dict['user'] = json_data['messages'][i+1]['sender_name']
        
        
        #removes reactions and links
        if not(('reacted' in this_dict['prompt'] and 'to your message' in this_dict['prompt']) or \
        ('reacted' in this_dict['response'] and 'to your message' in this_dict['response'])) and \
        '\u200d♂️' not in this_dict['response'] and '\u200d♂️' not in this_dict['prompt'] and \
        'https' not in this_dict['response'] and 'https' not in this_dict['prompt'] and \
        '.com' not in this_dict['response'] and '.com' not in this_dict['prompt']:
            prompt_response.append(this_dict)

In [31]:
prompt_response[:5]

[{'prompt': 'I’ll try',
  'response': "Can one of you guys download your facebook messenger data and upload the json file for this groupchat to the github repo? I can't download the 3 month one for some reason",
  'user': 'Ester Tsai'},
 {'prompt': "Can one of you guys download your facebook messenger data and upload the json file for this groupchat to the github repo? I can't download the 3 month one for some reason",
  'response': '😂',
  'user': 'Samuel Chu'},
 {'prompt': '😂',
  'response': 'keep sending texts and we will have better data 😳😳',
  'user': 'Ester Tsai'},
 {'prompt': 'Spam with what lol', 'response': 'lol ya', 'user': 'Samuel Chu'},
 {'prompt': 'lol ya',
  'response': 'you can spam the chat right now for data haha',
  'user': 'Ester Tsai'}]

In [33]:
with open('prompt_response.jsonl', 'w') as json_file:
    for entry in prompt_response:
        json.dump(entry, json_file)
        json_file.write('\n')

In [28]:
junk_messages = ['reacted', 'X:1759 T:F\"ur Elise T:Bagatelle', 'Counter({50:']

In [17]:
with open('data/cleaned.txt', 'w+') as f:
    for d in json_data['messages'][::-1]:
        if 'content' in d:
            message = d['content']
            if np.all([junk not in message for junk in junk_messages]):
                cleaned = message.replace('\n', ' ').encode('latin1').decode('utf-8')
                cleaned = '<start> ' + cleaned + ' <end> '
                f.write(cleaned)

In [14]:
with open('data/cleaned_with_name.txt', 'w+') as f:
    for d in json_data['messages'][::-1]:
        if 'content' in d:
            message = d['content']
            if np.all([junk not in message for junk in junk_messages]):
                sender = d["sender_name"].split(' ')[0].replace('Kong', 'Winfrey')
                cleaned = message.replace('\n', ' ').encode('latin1').decode('utf-8')
                cleaned = f'<start> {sender}: ' + cleaned + ' <end> '
                f.write(cleaned)